In [1]:
# -- FOR MODEL TRAINING --
import torch
from ultralytics import YOLO

In [2]:
device = ("cuda" if torch.cuda.is_available()
          else "mps" if torch.backends.mps.is_available()
          else "cpu")
device

'mps'

In [3]:
# -- CONFIG --
MODEL_SIZE = 'yolov8s.pt'
EPOCHS = 200
BATCH_SIZE = 32
IMG_SIZE = 1024
PATIENCE = 30

In [ ]:
# ----- TRAINING -----
model = YOLO(MODEL_SIZE)

model.train(
    data="config.yml",
    imgsz=IMG_SIZE,
    epochs=EPOCHS,
    batch=BATCH_SIZE,
    device=device,
    
    # Learning rate and optimization
    lr0=0.01,           # Initial learning rate
    lrf=0.01,           # Final learning rate (lr0 * lrf)
    momentum=0.937,     # SGD momentum or Adam beta1
    weight_decay=0.0005, # Optimizer weight decay
    warmup_epochs=3.0,   # Warmup epochs
    warmup_momentum=0.8, # Warmup initial momentum
    warmup_bias_lr=0.1,  # Warmup initial bias lr
    optimizer='auto',    # Optimizer: SGD, Adam, AdamW, NAdam, RAdam, RMSProp
    
    # Augmentation (adjusted for thermal images)
    hsv_h=0.01,         # Image HSV-Hue augmentation (fraction) - REDUCED for thermal
    hsv_s=0.5,          # Image HSV-Saturation augmentation (fraction) - REDUCED for thermal
    hsv_v=0.4,          # Image HSV-Value augmentation (fraction)
    degrees=10.0,       # Image rotation (+/- deg) - good for thermal
    translate=0.1,      # Image translation (+/- fraction)
    scale=0.5,          # Image scale (+/- gain)
    shear=5.0,          # Image shear (+/- deg)
    perspective=0.0,    # Image perspective (+/- fraction), range 0-0.001
    flipud=0.0,         # Image flip up-down (probability)
    fliplr=0.5,         # Image flip left-right (probability)
    mosaic=1.0,         # Image mosaic (probability) - good for dataset diversity
    mixup=0.1,          # Image mixup (probability) - helps with generalization
    copy_paste=0.0,     # Segment copy-paste (probability) - disable for detection
    
    # Training settings
    close_mosaic=10,    # Disable mosaic augmentation for final N epochs
    multi_scale=False,  # Set to True for multi-scale training (slower but better)
    cos_lr=True,       # Cosine LR scheduler (recommended: True for longer training)
    patience=30,        # Early stopping patience (epochs)
    save=True,          # Save train checkpoints and predict results
    save_period=-1,     # Save checkpoint every x epochs (disabled when <1)
    
    # Validation
    val=True,           # Validate/test during training
    split='val',        # Dataset split to use for validation: val, test, or train
    
    # Other important parameters
    cache=False,        # Cache images in memory/disk for faster training (True/ram/disk)
    workers=8,          # Max dataloader workers (per RANK if DDP)
    project=None,       # Project name (default: runs/detect)
    name='train',       # Experiment name (default: train, train2, train3, ...)
    exist_ok=True,      # Existing project/name ok, do not increment
    pretrained=True,    # Whether to use pretrained weights
    verbose=True,       # Verbose output
    seed=0,             # Random seed for reproducibility
    
    # Loss weights (fine-tuning these can help)
    box=7.5,            # Box loss gain
    cls=0.5,            # Class loss gain
    dfl=1.5,            # DFL loss gain
    
    # Advanced
    amp=True,           # Automatic Mixed Precision (AMP) training
    fraction=1.0,       # Dataset fraction to train on
    profile=False,      # Profile ONNX and TensorRT speeds during validation
    freeze=None,        # Freeze layers: backbone=10, first3=0 1 2
)

In [ ]:
# ----- Evaluation -----

metrics = model.val()
metrics = vars(metrics)

for k,v in metrics.items():
    print(f"{k}: {v}")

In [ ]:
# ----- Inference -----

model = YOLO("runs/detect/train/weights/best.pt")
results = model.predict(source="images/val",
                        imgsz=1280, 
                        conf=0.5,
                        iou=0.35,
                        save=True,
                        max_det=1000,
                        save_txt=True,
                        save_conf=True, 
                        device=device)
print(f"Inference saved to: {results[0].save_dir}")

In [ ]:
# --- Save model and export
model.export(format="onnx")